In [ ]:
%load_ext autoreload

In [ ]:
dataname="synthetic"
gpuid=0

patch_size=224 #should match the value used to train the network

In [ ]:
import torch
print(torch.__version__)
import torchvision
print(torchvision.__version__)

In [ ]:
print(torch.cuda.get_device_properties(gpuid))
torch.cuda.set_device(gpuid)
device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tables
import math

import torch
from torchvision.models import DenseNet
from torch.autograd import Variable
from torchvision import transforms

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
%cd My\ Drive/PytorchDigitalPathology/google-collab-version/PytorchDigitalPathology/visualization_densenet/

In [ ]:
import sys
sys.path.append('./pytorch-cnn-visualizations/src/')
from misc_functions import convert_to_grayscale

In [ ]:
checkpoint = torch.load(f"{dataname}_densenet_best_model.pth",  map_location=lambda storage, loc: storage) #load checkpoint to CPU and then put to dev

In [ ]:
#load the model, note that the paramters are coming from the checkpoint, since the architecture of the model needs to exactly match the weights saved

model = DenseNet(growth_rate=checkpoint["growth_rate"], block_config=checkpoint["block_config"],
                 num_init_features=checkpoint["num_init_features"], bn_size=checkpoint["bn_size"], drop_rate=checkpoint["drop_rate"], num_classes=checkpoint["n_classes"]).to(device)

print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")
model.load_state_dict(checkpoint["model_dict"])

In [ ]:
phase="val"
db=tables.open_file(f"./{dataname}_{phase}.pytable")

In [ ]:
img_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(size=(patch_size,patch_size),pad_if_needed=True), #these need to be in a reproducible order, first affine transforms and then color
    transforms.ToTensor()
    ])

In [ ]:
imgid=19

img = db.root.imgs[imgid,::]
#label = torch.tensor(db.root.labels[imgid])
label = torch.tensor(np.array(db.root.labels[imgid]))

img = img[:,:,None].repeat(3,axis=2) #convert to 3 channel

timg=img_transform(img)

plt.imshow(img)

In [ ]:
model.eval()

timg = timg.to(device)  # [1, 3, H, W]
label = label.type('torch.LongTensor').to(device)  # [1] with class indices (0, 1)

output = model(timg[None,::])  
output=output.detach().squeeze().cpu().numpy() #get output and pull it to CPU
    
predlabel=np.argmax(output)
print(f"class vals: {output}")
print(f"actual class: {label}")
print(f"predicted class: {predlabel}")

In [ ]:
def rescale_grads(map,gradtype="all"):
    if(gradtype=="pos"):    #positive
        map = (np.maximum(0, map) / map.max())
    elif gradtype=="neg":
        map = (np.maximum(0, -map) / -map.min())
    else:
        map = map - map.min()
        map /= map.max()
    return map

In [ ]:
from vanilla_backprop import VanillaBackprop
VBP = VanillaBackprop(model,device)
vanilla_grads = VBP.generate_gradients(timg[None,::], label)
vanilla_grads=np.moveaxis(vanilla_grads,0,-1)

fig, ax = plt.subplots(3,3, figsize=(20,10))
ax = ax.flatten()

ax[0].set_title("original")
ax[0].imshow(img)

ax[1].set_title("post transform")
ax[1].imshow(np.moveaxis(timg.cpu().numpy().squeeze(),0,-1))

ax[3].set_title("all gradients")
ax[3].imshow(rescale_grads(vanilla_grads,gradtype="all"))

ax[4].set_title("positive gradients")
ax[4].imshow(rescale_grads(vanilla_grads,gradtype="pos"))

ax[5].set_title("negative gradients")
ax[5].imshow(rescale_grads(vanilla_grads,gradtype="neg"))

ax[6].set_title("all gradients grayscale")
ax[6].imshow(convert_to_grayscale(rescale_grads(vanilla_grads,gradtype="all")))

ax[7].set_title("positive gradients grayscale")
ax[7].imshow(convert_to_grayscale(rescale_grads(vanilla_grads,gradtype="pos")))

ax[8].set_title("negative gradients grayscale")
ax[8].imshow(convert_to_grayscale(rescale_grads(vanilla_grads,gradtype="neg")))

In [ ]:
from guided_backprop import GuidedBackprop
GB=GuidedBackprop(model,device)
gp_grads=GB.generate_gradients(timg[None,::], label)


gp_grads=np.moveaxis(gp_grads,0,-1)

fig, ax = plt.subplots(3,3, figsize=(20,10))
ax = ax.flatten()

ax[0].set_title("original")
ax[0].imshow(img)

ax[1].set_title("post transform")
ax[1].imshow(np.moveaxis(timg.cpu().numpy().squeeze(),0,-1))

ax[3].set_title("all gradients")
ax[3].imshow(rescale_grads(gp_grads,gradtype="all"))

ax[4].set_title("positive gradients")
ax[4].imshow(rescale_grads(gp_grads,gradtype="pos"))

ax[5].set_title("negative gradients")
ax[5].imshow(rescale_grads(gp_grads,gradtype="neg"))

ax[6].set_title("all gradients grayscale")
ax[6].imshow(convert_to_grayscale(rescale_grads(gp_grads,gradtype="all")))

ax[7].set_title("positive gradients grayscale")
ax[7].imshow(convert_to_grayscale(rescale_grads(gp_grads,gradtype="pos")))

ax[8].set_title("negative gradients grayscale")
ax[8].imshow(convert_to_grayscale(rescale_grads(gp_grads,gradtype="neg")))

In [ ]:
from gradcam import GradCam

nlayers=len(model.features._modules.items())-1

fig, ax = plt.subplots(math.ceil(nlayers/4),4, figsize=(20,10))
ax = ax.flatten()

for layer in range(nlayers):
    grad_cam = GradCam(model, device,target_layer=layer)
    cam = grad_cam.generate_cam(timg[None,::], label)
    ax[layer].imshow(cam)

In [ ]:
from gradcam import GradCam
from guided_gradcam import guided_grad_cam
from guided_backprop import GuidedBackprop

nlayers=len(model.features._modules.items())-1

fig, ax = plt.subplots(math.ceil(nlayers/4),4, figsize=(20,10))
ax = ax.flatten()

for layer in range(nlayers):
    #GradCam
    grad_cam = GradCam(model, device,target_layer=layer)
    cam = grad_cam.generate_cam(timg[None,::], label)
    
    #GuidedBackprop
    GBP = GuidedBackprop(model, device)
    guided_grads = GBP.generate_gradients(timg[None,::], label)
    
    # Guided Grad cam
    cam_gb = guided_grad_cam(cam, guided_grads)
    
    ax[layer].imshow(convert_to_grayscale(np.moveaxis(cam_gb,0,-1)))